In [22]:
import requests
from pathlib import Path
import subprocess
import signal

In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, '/home/julien/Software/Others/OpenStudio/src/cli/')
from test_measure_manager import *

In [3]:
port = get_open_port_for_serving()
URL = f"http://{HOST}:{port}"
URL

Using port=8090


'http://localhost:8090'

In [4]:
OPENSTUDIO_CLI = Path('/home/julien/Software/Others/OS-build/Products/openstudio')

In [54]:
p = subprocess.Popen([str(OPENSTUDIO_CLI), 'labs', 'measure', '-s', str(port)])

In [56]:
p.poll()

In [53]:
p.send_signal(signal.SIGINT)

In [57]:
r = requests.get(f"{URL}")
r.json()

{'my_measures_dir': '', 'status': 'running'}

In [58]:
r = requests.get(f"{URL}/internal_state")
r.raise_for_status()
assert r.json() == {'my_measures_dir': '', 'osm': [], 'status': 'running'}

In [59]:
r.json()

{'my_measures_dir': '', 'osm': [], 'status': 'running'}

In [60]:
my_measures_dir = str(Path('~/OpenStudio/Measures').expanduser())
r = requests.post(url=f"{URL}/set", json={"my_measures_dir": my_measures_dir})
r.raise_for_status()

In [61]:
r = requests.get(f"{URL}/internal_state")
r.raise_for_status()
assert r.json() == {'my_measures_dir': my_measures_dir, 'osm': [], 'status': 'running'}

In [62]:
r.json()

{'my_measures_dir': '/home/julien/OpenStudio/Measures',
 'osm': [],
 'status': 'running'}

In [63]:
import openstudio
OSM_PATH = Path('/media/DataExt4/Software/Others/OS-build3/model.osm')
OSM_PATH2 = Path('/media/DataExt4/Software/Others/OS-build3/model2.osm')

model = openstudio.model.Model()
model.save(str(OSM_PATH), True)
model = openstudio.model.Model()
model.save(str(OSM_PATH2), True)

True

In [64]:
openstudio.checksum(openstudio.toPath(str(OSM_PATH)))

'9FA60624'

In [65]:
r = requests.post(url=f"{URL}/get_model", json={"osm_path": str(OSM_PATH)})
r.raise_for_status()
r.json()

┌────────────────────────────────────────────────────────────────────────────────┐
│         The `labs` command is experimental - Do not use in production          │
└────────────────────────────────────────────────────────────────────────────────┘

MeasureUpdateOptions:
directoryPath=
update=false
update_all=false
compute_arguments_model=
run_tests=false
server_port=8090


Received GET request: GET / HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate, br
Connection: keep-alive
Host: localhost:8090
User-Agent: python-requests/2.28.1

uri=/
Received GET request: GET /internal_state HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate, br
Connection: keep-alive
Host: localhost:8090
User-Agent: python-requests/2.28.1

uri=/internal_state
Received POST request: POST /set HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate, br
Connection: keep-alive
Content-Length: 55
Content-Type: application/json
Host: localhost:8090
User-Agent: python-requests/2.28.1

Received GET request: GET /internal_

'OK, loaded model with checksum 9FA60624'

In [66]:
r = requests.get(f"{URL}/internal_state")
r.raise_for_status()
r.json()

{'my_measures_dir': '/home/julien/OpenStudio/Measures',
 'osm': [{'checksum': '9FA60624',
   'osm_path': '/media/DataExt4/Software/Others/OS-build3/model.osm'}],
 'status': 'running'}

In [18]:
r = requests.post(url=f"{URL}/get_model", json={"osm_path": str(OSM_PATH)})
r.raise_for_status()
r.json()

'OK'

In [40]:
r = requests.post(url=f"{URL}/get_model", json={"osm_path": str(OSM_PATH2)})
r.raise_for_status()
r.json()

E59D919B
Received POST request: POST /get_model HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate, br
Connection: keep-alive
Content-Length: 67
Content-Type: application/json
Host: localhost:8090
User-Agent: python-requests/2.28.1

Using cached model /media/DataExt4/Software/Others/OS-build3/model.osm
E59D919B
Received POST request: POST /get_model HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate, br
Connection: keep-alive
Content-Length: 67
Content-Type: application/json
Host: localhost:8090
User-Agent: python-requests/2.28.1

{"osm_path": "/media/DataExt4/Software/Others/OS-build3/model.osm"}Using cached model /media/DataExt4/Software/Others/OS-build3/model.osm
E59D919B
Received GET request: GET /internal_state HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate, br
Connection: keep-alive
Host: localhost:8090
User-Agent: python-requests/2.28.1

uri=/internal_state
Received POST request: POST /get_model HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate, br
Connection: keep-aliv

'OK, loaded model with checksum 608AB8B6'

In [ ]:
r = requests.post(url=f"{URL}/download_bcl_measure", json={"": ""})
assert r.status_code == 400
assert r.json() == 'Missing the uid in the post data'

In [ ]:
r = requests.post(url=f"{URL}/download_bcl_measure", json={"uid": "baduuid"})
assert r.status_code == 400
assert r.json() == "Cannot find measure with uid='baduuid'"

In [ ]:
BCL_DIR = str(Path('~/BCL').expanduser())
TDB_UID = '8890787b-8c25-4dc8-8641-b6be1b6c2357'
OPENSTUDIO_RESULTS_UID = 'a25386cd-60e4-46bc-8b11-c755f379d916'

In [ ]:
r = requests.post(url=f"{URL}/download_bcl_measure", json={"uid": TDB_UID})
r.raise_for_status()

In [ ]:
r = requests.post(url=f"{URL}/bcl_measures")
r.raise_for_status()

In [ ]:
r.json()

In [ ]:
r = requests.post(url=f"{URL}/reset")
r.raise_for_status()

In [ ]:
for endpoint in ['update_measures', 'compute_arguments', 'create_measure', 'duplicate_measure']:
    r = requests.post(url=f"http://{URL}/{endpoint}")
    assert r.status_code == 501

In [ ]:
r.json()